In [41]:
import json
import psycopg2
import pandas as pd

In [51]:
df = pd.read_csv('watersucia.csv', sep=';')

In [64]:
import json
import psycopg2
import pandas as pd

def read_db_config(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def create_database(config, db_name):
    try:
        connection = psycopg2.connect(**config)
        connection.autocommit = True
        cursor = connection.cursor()
        cursor.execute(f"CREATE DATABASE {db_name};")
        print(f"Base de datos {db_name} creada exitosamente!")
        cursor.close()
    except psycopg2.Error as e:
        print(f"No se puede crear la base de datos: {e}")
    finally:
        if connection:
            connection.close()

def create_table(config, db_name):
    try:
        config['dbname'] = db_name
        connection = psycopg2.connect(**config)
        cursor = connection.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS water_table (
                "Año" DATE,
                "NombreDepartamento" VARCHAR(255),
                "Div_dpto" INT,
                "NombreMunicipio" VARCHAR(255),
                "Divi_muni" INT,
                "IrcaMinimo2" FLOAT,
                "IrcaMaximo" FLOAT,
                "IrcaPromedio" FLOAT,
                "NombreParametroAnalisis2" VARCHAR(255),
                "MuestrasEvaluadas" INT,
                "NumeroParametrosMinimo" INT,
                "NumeroParametrosMaximo" INT,
                "NumeroParametrosPromedio" INT,
                "NumeroMuestras" INT,
                "MuestrasNoAptas" INT,
                "PorcentajeNoAptas" FLOAT,
                "ResultadoMinimo" FLOAT,
                "ResultadoMaximo" FLOAT,
                "ResultadoPromedio" FLOAT,
                "Codigo" VARCHAR(255)
            );
        """)
        print("Tabla creada con éxito.")
        connection.commit()
        cursor.close()
    except psycopg2.Error as e:
        print(f"No se puede crear la tabla: {e}")
    finally:
        if connection:
            connection.close()

def insertar_datos(df, config, db_name):
    try:
        config['dbname'] = db_name
        connection = psycopg2.connect(**config)
        cursor = connection.cursor()
        query = """
        INSERT INTO water_table ("Año", "NombreDepartamento", "Div_dpto", "NombreMunicipio", "Divi_muni", "IrcaMinimo2", "IrcaMaximo", "IrcaPromedio", "NombreParametroAnalisis2", "MuestrasEvaluadas", "NumeroParametrosMinimo", "NumeroParametrosMaximo", "NumeroParametrosPromedio", "NumeroMuestras", "MuestrasNoAptas", "PorcentajeNoAptas", "ResultadoMinimo", "ResultadoMaximo", "ResultadoPromedio", "Codigo")
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        for index, row in df.iterrows():
            # Convertir el año a formato DATE
            year = pd.to_datetime(str(row["Año"]), format='%Y').date()
            # Reemplazar comas por puntos en los valores decimales
            def convert_decimal(value):
                return float(str(value).replace(',', '.')) if pd.notnull(value) else None
            
            data = (
                year,
                row["NombreDepartamento"],
                row["Div_dpto"],
                row["NombreMunicipio"],
                row["Divi_muni"],
                convert_decimal(row["IrcaMinimo2"]),
                convert_decimal(row["IrcaMaximo"]),
                convert_decimal(row["IrcaPromedio"]),
                row["NombreParametroAnalisis2"],
                row["MuestrasEvaluadas"],
                row["NumeroParametrosMinimo"],
                row["NumeroParametrosMaximo"],
                row["NumeroParametrosPromedio"],
                row["NumeroMuestras"],
                row["MuestrasNoAptas"],
                convert_decimal(row["PorcentajeNoAptas"]),
                convert_decimal(row["ResultadoMinimo"]),
                convert_decimal(row["ResultadoMaximo"]),
                convert_decimal(row["ResultadoPromedio"]),
                row["Codigo"]
            )
            cursor.execute(query, data)
        connection.commit()
        print("Datos insertados exitosamente")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()



if __name__ == "__main__":
    config = read_db_config('db_config.json')
    db_name = 'db_water'
    create_database(config, db_name)
    create_table(config, db_name)
    insertar_datos(df, config, db_name)


No se puede crear la base de datos: la base de datos «db_water» ya existe

Tabla creada con éxito.
Datos insertados exitosamente
